In [1]:
%%capture
!git clone https://github.com/HackerBschor/SofteningQueryEvaluation
%cd SofteningQueryEvaluation

!pip3 install faiss-gpu-cu12
!pip3 install pgvector
!pip3 install kagglehub

In [2]:
from huggingface_hub import notebook_login
notebook_login()

# Evaluate Similarity Grouping


In [3]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel
from models.semantic_validation import LLaMAValidationModel

from db.operators import Dummy, Select
from db.criteria import SoftEqual
from db.structure import Column, Constant
import kagglehub

from evaluation.util import calculate_metrics, calc_bleu

import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
# Models
# meta-llama/Llama-3.2-3B-Instruct
# meta-llama/Meta-Llama-3-8B-Instruct

stem = SentenceTransformerEmbeddingModel(ModelMgr())
lsv = LLaMAValidationModel(ModelMgr(), model_path = "meta-llama/Llama-3.2-3B-Instruct")
# lsv = DeepSeekValidationModel(m)

model-00001-of-00002.safetensors:  57%|#####6    | 2.81G/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [14]:
ZERO_SHOT_SYSTEM_PROMPT = "You are a validator. Respond with \"no\" and \"yes\" only!"
ZERO_SHOT_PROMPTING_TEMPLATE = 'Does "{}" describes "{}"'

random_state = 420
max_number_categories = 10

target = "category_2"

In [7]:
path = kagglehub.dataset_download("lakritidis/product-classification-and-categorization")

df = pd.read_csv(f"{path}/pricerunner_aggregate.csv", header=None, index_col=0)
df.drop(columns=[2, 3, 5], inplace=True)
df.columns = ["product", "category_1", "category_2"]
df.head()

100%|██████████| 22.8M/22.8M [00:00<00:00, 85.4MB/s]

Extracting files...


,product,category_1,category_2
0,,,
1,apple iphone 8 plus 64gb silver,Apple iPhone 8 Plus 64GB,Mobile Phones
2,apple iphone 8 plus 64 gb spacegrau,Apple iPhone 8 Plus 64GB,Mobile Phones
3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,Apple iPhone 8 Plus 64GB,Mobile Phones
4,apple iphone 8 plus 64gb space grey,Apple iPhone 8 Plus 64GB,Mobile Phones
5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,Apple iPhone 8 Plus 64GB,Mobile Phones


In [8]:
categories = pd.Series(df[target].unique())
categories = categories.sample(min(len(categories), max_number_categories)).tolist()

print(len(categories)), print(categories)

10
['TVs', 'Washing Machines', 'Dishwashers', 'Fridge Freezers', 'Microwaves', 'Digital Cameras', 'CPUs', 'Fridges', 'Freezers', 'Mobile Phones']


(None, None)

In [16]:
df_reduced = pd.concat([df[df[target] == category].sample(max_number_categories, random_state=random_state) for category in categories])

gt = {(x["product"], x[target]) for key, x in df_reduced.iterrows()}
print(str(gt)[0:100], "...", len(gt))

{('russell hobbs rhbm3201mb 32l built in digital 1000w combination microwave matte black', 'Microwav ... 100


In [17]:
overall_result = {}

def evaluate(method, threshold, system_prompt=ZERO_SHOT_SYSTEM_PROMPT, prompt_template=ZERO_SHOT_PROMPTING_TEMPLATE):
    pred = []
    runtimes = []
    for category in tqdm(categories):
        d = Dummy("products", ["product"], [(x[0], ) for x in gt])
        s = Select(d, SoftEqual(Column("product"),Constant(category), method=method, em=stem, sv=lsv, threshold=threshold, zfs_system_prompt=system_prompt, zfs_prompt_template = prompt_template))

        tic = time.time()
        result = s.open().fetch_all()
        toc = time.time()

        pred.extend([(x["product"], category) for x in result])
        runtimes.append(toc - tic)

    scores = calculate_metrics(gt, set(pred), np.mean(runtimes))

    print(method, threshold, scores["F1 Score"])

    return scores, pred

In [18]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("threshold", thresh)
    overall_result[("threshold", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 10/10 [00:13<00:00,  1.39s/it]


threshold 0.1 0.2680115273775216


100%|██████████| 10/10 [00:12<00:00,  1.27s/it]


threshold 0.2 0.5478547854785478


100%|██████████| 10/10 [00:12<00:00,  1.27s/it]


...

threshold 0.4 0.6067415730337079


100%|██████████| 10/10 [00:12<00:00,  1.27s/it]


threshold 0.5 0.21666666666666667


100%|██████████| 10/10 [00:12<00:00,  1.27s/it]

threshold 0.6 0


In [19]:
res, pred = evaluate("zero-few-shot", None)
overall_result[("zero-few-shot", None)] = res

100%|██████████| 10/10 [00:47<00:00,  4.76s/it]

zero-few-shot None 0.6103896103896103


In [21]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("both", thresh)
    overall_result[("both", thresh)] = res

100%|██████████| 10/10 [00:39<00:00,  4.00s/it]


both 0.1 0.6103896103896103


100%|██████████| 10/10 [00:22<00:00,  2.21s/it]


both 0.2 0.6013071895424837


100%|██████████| 10/10 [00:18<00:00,  1.83s/it]


...

both 0.8 0


100%|██████████| 10/10 [00:12<00:00,  1.27s/it]


both 0.9 0


100%|██████████| 10/10 [00:12<00:00,  1.28s/it]

both 1.0 0


In [22]:
for key in tqdm(overall_result):
    scores_bleu = calc_bleu(gt, overall_result[key]["pred"])
    for score_bleu in scores_bleu:
        overall_result[key][score_bleu] = scores_bleu[score_bleu]

100%|██████████| 17/17 [00:48<00:00,  2.88s/it]


In [26]:
df_results = pd.DataFrame([{"method": k[0], "threshold": k[1]} | v for k,v in overall_result.items()])
df_results

,method,threshold,Precision,Recall,F1 Score,tp,fn,fp,runtime,pred,bleu1,bleu2,bleu3,bleu4
0,threshold,0.1,0.156566,0.93,0.268012,93,7,501,1.384712,{(intel xeon e7420 processor 2.13 ghz box 8 mb...,0.987544,0.986159,0.984264,0.982120
1,threshold,0.2,0.408867,0.83,0.547855,83,17,120,1.273098,{(russell hobbs rhbm3201mb 32l built in digita...,0.908404,0.892725,0.877921,0.862979
2,threshold,0.3,0.628099,0.76,0.687783,76,24,45,1.269490,{(russell hobbs rhbm3201mb 32l built in digita...,0.839222,0.819443,0.800086,0.784812
3,threshold,0.4,0.692308,0.54,0.606742,54,46,24,1.271379,{(russell hobbs rhbm3201mb 32l built in digita...,0.701566,0.671405,0.638418,0.611531
4,threshold,0.5,0.650000,0.13,0.216667,13,87,7,1.269422,"{(belling bel fw714 whi washing machine white,...",0.378740,0.306038,0.265724,0.242154
5,threshold,0.6,0.000000,0.00,0.000000,0,100,1,1.273246,{(fridgemaster mtz55183ff tall frost free free...,0.123497,0.061672,0.045768,0.035064
6,zero-few-shot,NaN,0.870370,0.47,0.610390,47,53,7,4.759537,{(philips 55hfl7011t/12 55 inch signature prof...,0.658969,0.614885,0.564154,0.533961
7,both,0.1,0.870370,0.47,0.610390,47,53,7,3.995259,{(philips 55hfl7011t/12 55 inch signature prof...,0.658969,0.614885,0.564154,0.533961
8,both,0.2,0.867925,0.46,0.601307,46,54,7,2.206443,{(philips 55hfl7011t/12 55 inch signature prof...,0.651318,0.606850,0.555453,0.524560
9,both,0.3,0.875000,0.42,0.567568,42,58,6,1.830830,{(philips 55hfl7011t/12 55 inch signature prof...,0.625200,0.577351,0.524159,0.489745


In [28]:
df_results.to_pickle("Products_mpnetBaseV2_LLama3B.pkl")
df_results.drop(columns=["pred"]).to_csv("Products_mpnetBaseV2_LLama3B.csv")